## Support Vector Classifier for genre, based on 4 simple features

Trained with k fold cross validation and a loop over multple shufflings of the data.
Features used are the meadians of the simple feature vectors for each song.

Choose between linear and RBF kernel and between taking the mean or the median over the data. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
# Data import

genres = ["classical", "electronic", "pop", "rock"]

ZC = np.load('../preprocessing/SimpleFeatures/zeroCrossings_frame200ms_hop100ms.npy')
SC = np.load('../preprocessing/SimpleFeatures/spectralCentroid_frame200ms_hop100ms.npy')
SV = np.load('../preprocessing/SimpleFeatures/spectralVariance_frame200ms_hop100ms.npy')
ST = np.load('../preprocessing/SimpleFeatures/staticTempoLibrosa.npy')

In [3]:
# Take the median over all frames for each song to have comparable dimensions

ZC = np.nanmedian(ZC, axis=1)
SC = np.nanmedian(SC, axis=1)
SV = np.nanmedian(SV, axis=1)

In [4]:
# OR take the mean in a certain interval

# ZC = np.mean(ZC[:, 100:200], axis=1)
# SC = np.mean(SC[:, 100:200], axis=1)
# SV = np.mean(SV[:, 2:4], axis=1)

In [5]:
# Make dataframe

features_dict_medians = {'ZC': ZC,
                         'SC': SC,
                         'SV': SV,
                         'StaticTempo': ST}
features = pd.DataFrame(data=features_dict_medians)

ones = np.ones(100, dtype='int')
labels_dict = {'label': np.concatenate((ones, ones*2, ones*3, ones*4))}
labels = pd.DataFrame(labels_dict)

In [6]:
# Shuffle

def shuffleData(features, labels):
    n_data = len(features)
    idx = np.random.permutation(n_data)
    features, labels = features.reindex(idx), labels.reindex(idx)
    return features, labels

In [7]:
# K fold Crossvalidation

def kFoldConfusionMatrix(k, features, labels):

    kfold_train_metrics = []
    kfold_test_metrics = []

    confusion_Matrix = np.zeros((4,4))

    # # Drop NaN values
    # NaNidx = np.where(features.isnull().any(axis=1).tolist())[0]
    # # NaNidx = np.isnan(features)[np.isnan(features) == 1].stack().index.tolist()

    # total_input = features.drop(NaNidx)
    # total_labels = labels.drop(NaNidx)

    total_input = features
    total_labels = labels

    cv = KFold(n_splits=k)

    for train_index, test_index in cv.split(total_input):

        train_df, train_labels = total_input.iloc[train_index], total_labels.iloc[train_index]
        test_df, test_labels = total_input.iloc[test_index], total_labels.iloc[test_index]

        # Standardizing data
        mean = train_df.mean()
        std = train_df.std()

        train_df = (train_df - mean) / std
        test_df = (test_df - mean) / std

        # Classifier
        clf = svm.SVC(decision_function_shape='ovo', kernel='linear') # One versus one SVM classifier with RBF kernel
        clf.fit(train_df, np.squeeze(train_labels.to_numpy()))

        kfold_train_metrics.append(clf.score(train_df, train_labels))
        kfold_test_metrics.append(clf.score(test_df, test_labels))

        confusion_Matrix += confusion_matrix(test_labels, clf.predict(test_df), labels=[1,2,3,4])
        
    return confusion_Matrix

In [8]:
# Doing the actual training of the support vector classifier over multple rounds of shuffeling

confusion_Matrix = np.zeros((4,4))

N_shuffles = 20
k = 10
accuracies = []

for Siter in range(N_shuffles):
    features, labels = shuffleData(features, labels)
    confusion_Matrix += kFoldConfusionMatrix(k, features, labels)
    
    accuracies.append(np.sum(np.eye(4)*confusion_Matrix)/np.sum(confusion_Matrix))

In [9]:
# Confusion matrix
confusion_Matrix

array([[1641.,  106.,  116.,  137.],
       [ 790.,  275.,  445.,  490.],
       [ 239.,  226.,  599.,  936.],
       [ 234.,  148.,  778.,  840.]])

In [ ]:
CM = ConfusionMatrixDisplay(confusion_Matrix)
CM.plot()

In [10]:
print(f'Accuracy: {np.mean(accuracies)} +- {np.std(accuracies)}')

Accuracy: 0.4240440872986878 +- 0.004481786797524654
